In [37]:
import pandas as pd

In [38]:
import numpy as np

### Carregar dados

In [39]:
jogos = pd.read_csv("data/jogos.csv")

In [40]:
jogos.head()

,Ano,Mandante,Visitante,Placar_mandante,Placar_visitante
0,2017,Vasco,Coritiba,1,1
1,2017,Flamengo,Bahia,4,1
2,2017,Sport Recife,Santos,1,1
3,2017,Palmeiras,Ponte Preta,2,0
4,2017,Vitória,Atlético-PR,2,3


In [41]:
jogos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 5 columns):
Ano                 1431 non-null int64
Mandante            1431 non-null object
Visitante           1431 non-null object
Placar_mandante     1431 non-null int64
Placar_visitante    1431 non-null int64
dtypes: int64(3), object(2)
memory usage: 56.0+ KB


### Remover espaços em branco dos nomes dos times

In [42]:
jogos_obj = jogos.select_dtypes(['object'])
jogos[jogos_obj.columns] = jogos_obj.apply(lambda x: x.str.strip())

### Lista de times participantes

In [43]:
times = list(jogos["Visitante"].unique())

In [44]:
times.sort()

In [45]:
times

['América-MG',
 'Atlético-GO',
 'Atlético-MG',
 'Atlético-PR',
 'Avaí',
 'Bahia',
 'Botafogo',
 'Chapecoense',
 'Corinthians',
 'Coritiba',
 'Criciúma',
 'Cruzeiro',
 'Figueirense',
 'Flamengo',
 'Fluminense',
 'Goias',
 'Grêmio',
 'Internacional',
 'Joinville',
 'Palmeiras',
 'Ponte Preta',
 'Santa Cruz',
 'Santos',
 'Sport Recife',
 'São Paulo',
 'Vasco',
 'Vitória']

In [46]:
len(times)

27

### Criar coluna com resultado

In [47]:
def resultado(row):
    if (row["Placar_mandante"] > row["Placar_visitante"]):
        return "MANDANTE"
    elif (row["Placar_mandante"] < row["Placar_visitante"]):
        return "VISITANTE"
    else:
        return "EMPATE"

In [48]:
jogos['Resultado'] = jogos.apply (lambda row: resultado(row),axis=1)

In [49]:
jogos.head()

,Ano,Mandante,Visitante,Placar_mandante,Placar_visitante,Resultado
0,2017,Vasco,Coritiba,1,1,EMPATE
1,2017,Flamengo,Bahia,4,1,MANDANTE
2,2017,Sport Recife,Santos,1,1,EMPATE
3,2017,Palmeiras,Ponte Preta,2,0,MANDANTE
4,2017,Vitória,Atlético-PR,2,3,VISITANTE


### Totais de resultados por ano

In [50]:
jogos.groupby(["Ano", "Resultado"])[["Ano"]].count()

Ano
Ano  Resultado     
2014 EMPATE      92
     MANDANTE   197
     VISITANTE   91
2015 EMPATE      91
     MANDANTE   200
     VISITANTE   89
2016 EMPATE      95
     MANDANTE   202
     VISITANTE   83
2017 EMPATE      77
     MANDANTE   124
     VISITANTE   90

### Totais por resultado

In [51]:
totais_resultados = jogos.groupby("Resultado").size()

In [52]:
totais_resultados

Resultado
EMPATE       355
MANDANTE     723
VISITANTE    353
dtype: int64

In [53]:
total_jogos = totais_resultados.sum()

In [54]:
totais_resultados.keys()

Index(['EMPATE', 'MANDANTE', 'VISITANTE'], dtype='object', name='Resultado')

In [55]:
totais_resultados.values

array([355, 723, 353])

In [56]:
resultados_DF = pd.DataFrame({'Resultados': totais_resultados.keys(), 'Totais': totais_resultados.values})

In [57]:
resultados_DF

,Resultados,Totais
0,EMPATE,355
1,MANDANTE,723
2,VISITANTE,353


In [58]:
def percentual(valor, total):
    return round(valor/total, 3)

In [59]:
resultados_DF['Perc'] = resultados_DF.apply (lambda row: percentual(row["Totais"], total_jogos),axis=1)

In [60]:
resultados_DF

,Resultados,Totais,Perc
0,EMPATE,355,0.248
1,MANDANTE,723,0.505
2,VISITANTE,353,0.247


### Calcular resultados de um time

In [61]:
def resultadosTime(time, casa, resultado):
    return len(jogos[(jogos[casa] == time) & (jogos["Resultado"] == resultado)]) 

In [62]:
resultados_time_DF = pd.DataFrame({'Times': times })

In [63]:
resultados_time_DF["vitorias_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Mandante", "MANDANTE"),axis=1)
resultados_time_DF["empates_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Mandante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_casa"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Mandante", "VISITANTE"),axis=1)
resultados_time_DF["vitorias_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Visitante", "VISITANTE"),axis=1)
resultados_time_DF["empates_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Visitante", "EMPATE"),axis=1)
resultados_time_DF["derrotas_fora"] = resultados_time_DF.apply (lambda row: resultadosTime(row["Times"], "Visitante", "MANDANTE"),axis=1)

In [64]:
resultados_time_DF.head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora
0,América-MG,7,3,9,0,4,15
1,Atlético-GO,3,3,9,4,2,8
2,Atlético-MG,42,13,17,23,24,24


In [65]:
resultados_time_DF.eval("jogos_casa = vitorias_casa + empates_casa + derrotas_casa", inplace=True)
resultados_time_DF.eval("jogos_fora = vitorias_fora + empates_fora + derrotas_fora", inplace=True)
resultados_time_DF.eval("TOTAL = jogos_fora + jogos_fora", inplace=True)

In [66]:
resultados_time_DF.head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,TOTAL
0,América-MG,7,3,9,0,4,15,19,19,38
1,Atlético-GO,3,3,9,4,2,8,15,14,28
2,Atlético-MG,42,13,17,23,24,24,72,71,142


In [67]:
def aproveitamentoMandante(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    return (3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante)

In [68]:
resultados_time_DF["aprov_casa"] = resultados_time_DF.apply (lambda row: aproveitamentoMandante(row),axis=1)
resultados_time_DF.head(3)


,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,TOTAL,aprov_casa
0,América-MG,7,3,9,0,4,15,19,19,38,0.421053
1,Atlético-GO,3,3,9,4,2,8,15,14,28,0.266667
2,Atlético-MG,42,13,17,23,24,24,72,71,142,0.643519


In [69]:
def aproveitamentoVisitante(row):
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"]) / (3*total_jogos_visitante)

In [70]:
resultados_time_DF["aprov_fora"] = resultados_time_DF.apply (lambda row: aproveitamentoVisitante(row),axis=1)
resultados_time_DF.head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,TOTAL,aprov_casa,aprov_fora
0,América-MG,7,3,9,0,4,15,19,19,38,0.421053,0.070175
1,Atlético-GO,3,3,9,4,2,8,15,14,28,0.266667,0.333333
2,Atlético-MG,42,13,17,23,24,24,72,71,142,0.643519,0.436620


In [71]:
def aproveitamentoTotal(row):
    total_jogos_mandante = row["vitorias_casa"] + row["empates_casa"] + row["derrotas_casa"]
    total_jogos_visitante = row["vitorias_fora"] + row["empates_fora"] + row["derrotas_fora"]
    return (3*row["vitorias_fora"] + row["empates_fora"] + 3*row["vitorias_casa"] + row["empates_casa"]) / (3*total_jogos_mandante + 3*total_jogos_visitante)

In [72]:
resultados_time_DF["aprov_total"] = resultados_time_DF.apply (lambda row: aproveitamentoTotal(row),axis=1)
resultados_time_DF.sort_values(["aprov_total"], ascending=[0]).head(3)

,Times,vitorias_casa,empates_casa,derrotas_casa,vitorias_fora,empates_fora,derrotas_fora,jogos_casa,jogos_fora,TOTAL,aprov_casa,aprov_fora,aprov_total
8,Corinthians,47,19,6,28,20,23,72,71,142,0.740741,0.488263,0.615385
11,Cruzeiro,39,21,11,27,14,31,71,72,144,0.647887,0.439815,0.543124
22,Santos,49,11,11,17,23,32,71,72,144,0.741784,0.342593,0.540793


In [73]:
resultados_time_DF = resultados_time_DF[["Times", "jogos_casa", "jogos_fora", "TOTAL", "aprov_casa", "aprov_fora", "aprov_total"]].sort_values(["aprov_total"], ascending=[0])

In [74]:
def golsMandante(time):
    return jogos[jogos["Mandante"] == time][["Placar_mandante"]].sum();

In [75]:
def golsVisitante(time):
    return jogos[jogos["Visitante"] == time][["Placar_visitante"]].sum();

In [79]:
resultados_time_DF["gols_mandante"] = resultados_time_DF.apply (lambda row: golsMandante(row["Times"]),axis=1)
resultados_time_DF["gols_visitante"] = resultados_time_DF.apply (lambda row: golsVisitante(row["Times"]),axis=1)

resultados_time_DF.eval("total_gols = gols_mandante + gols_visitante", inplace=True)
resultados_time_DF.eval("media_gols_casa = gols_mandante/jogos_casa", inplace=True)
resultados_time_DF.eval("media_gols_fora = gols_visitante/jogos_fora", inplace=True)
resultados_time_DF.eval("media_gols = (gols_mandante + gols_visitante)/TOTAL", inplace=True)

In [80]:
resultados_time_DF.head()


,Times,jogos_casa,jogos_fora,TOTAL,aprov_casa,aprov_fora,aprov_total,gols_mandante,gols_visitante,total_gols,media_gols_casa,media_gols_fora,media_gols
8,Corinthians,72,71,142,0.740741,0.488263,0.615385,123,84,207,1.708333,1.183099,1.457746
11,Cruzeiro,71,72,144,0.647887,0.439815,0.543124,119,75,194,1.676056,1.041667,1.347222
22,Santos,71,72,144,0.741784,0.342593,0.540793,126,65,191,1.774648,0.902778,1.326389
2,Atlético-MG,72,71,142,0.643519,0.436620,0.540793,120,90,210,1.666667,1.267606,1.478873
16,Grêmio,71,72,144,0.694836,0.388889,0.540793,100,71,171,1.408451,0.986111,1.187500


In [89]:
resultados_time_DF.sort_values(["media_gols"], ascending=[0])

,Times,jogos_casa,jogos_fora,TOTAL,aprov_casa,aprov_fora,aprov_total,gols_mandante,gols_visitante,total_gols,media_gols_casa,media_gols_fora,media_gols
2,Atlético-MG,72,71,142,0.643519,0.436620,0.540793,120,90,210,1.666667,1.267606,1.478873
8,Corinthians,72,71,142,0.740741,0.488263,0.615385,123,84,207,1.708333,1.183099,1.457746
19,Palmeiras,72,71,142,0.606481,0.431925,0.519814,111,88,199,1.541667,1.239437,1.401408
11,Cruzeiro,71,72,144,0.647887,0.439815,0.543124,119,75,194,1.676056,1.041667,1.347222
24,São Paulo,71,72,144,0.652582,0.365741,0.508159,118,75,193,1.661972,1.041667,1.340278
22,Santos,71,72,144,0.741784,0.342593,0.540793,126,65,191,1.774648,0.902778,1.326389
14,Fluminense,72,71,142,0.574074,0.338028,0.456876,116,69,185,1.611111,0.971831,1.302817
13,Flamengo,72,71,142,0.611111,0.403756,0.508159,114,69,183,1.583333,0.971831,1.288732
23,Sport Recife,72,71,142,0.634259,0.262911,0.449883,110,63,173,1.527778,0.887324,1.218310
26,Vitória,53,52,104,0.377358,0.358974,0.368254,69,56,125,1.301887,1.076923,1.201923


In [93]:
resultados_time_DF = resultados_time_DF[["Times", "TOTAL", "aprov_casa", "aprov_fora", "aprov_total", "media_gols_casa", "media_gols_fora", "media_gols"]]

In [96]:
resultados_time_DF.sort_values(["aprov_total"], ascending = [0])

,Times,TOTAL,aprov_casa,aprov_fora,aprov_total,media_gols_casa,media_gols_fora,media_gols
8,Corinthians,142,0.740741,0.488263,0.615385,1.708333,1.183099,1.457746
11,Cruzeiro,144,0.647887,0.439815,0.543124,1.676056,1.041667,1.347222
22,Santos,144,0.741784,0.342593,0.540793,1.774648,0.902778,1.326389
2,Atlético-MG,142,0.643519,0.436620,0.540793,1.666667,1.267606,1.478873
16,Grêmio,144,0.694836,0.388889,0.540793,1.408451,0.986111,1.187500
19,Palmeiras,142,0.606481,0.431925,0.519814,1.541667,1.239437,1.401408
13,Flamengo,142,0.611111,0.403756,0.508159,1.583333,0.971831,1.288732
24,São Paulo,144,0.652582,0.365741,0.508159,1.661972,1.041667,1.340278
17,Internacional,114,0.708333,0.292398,0.498525,1.517857,0.719298,1.105263
3,Atlético-PR,144,0.633803,0.300926,0.466200,1.436620,0.791667,1.104167
